In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import cv2

In [2]:
train_dir = 'train'

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def draw_labels(images, labels):
    labels = labels.argmax(1)

    font = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (0,28)
    fontScale = 1
    fontColor = (255,255,255)
    lineWidth = 2
    lineType = cv2.LINE_AA

    labeled_images = []
    for idx, img in enumerate(images):
        lbl = labels[idx]
        img = img.reshape((28, 28)) * 255
        img = cv2.resize(img, (256, 256))

        cv2.putText(
            img,
            str(lbl),
            bottomLeftCornerOfText, 
            font, 
            fontScale,
            fontColor,
            lineWidth,
            lineType
        )

        labeled_images.append(img.reshape((256, 256, 1)))
    
    return np.asarray(labeled_images)

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')        

In [5]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name='flat_images')
    x_image = tf.reshape(x, [-1,28,28,1], name='reshaped_images')
    y_target = tf.placeholder(tf.float32, shape=[None, 10], name='target_labels')

with tf.name_scope('conv_layer_1'):
    with tf.name_scope('weights'):
        W_conv1 = weight_variable([5, 5, 1, 29])
        variable_summaries(W_conv1)
    with tf.name_scope('biases'):
        b_conv1 = bias_variable([29])
        variable_summaries(b_conv1)
    with tf.name_scope('relu_maxpool'):
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

with tf.name_scope('conv_layer_2'):
    with tf.name_scope('weights'):
        W_conv2 = weight_variable([5, 5, 29, 50])
        variable_summaries(W_conv2)
    with tf.name_scope('biases'):
        b_conv2 = bias_variable([50])
        variable_summaries(b_conv2)
    with tf.name_scope('relu_maxpool'):
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)

with tf.name_scope('fc_layer'):
    with tf.name_scope('weights'):
        W_fc1 = weight_variable([7 * 7 * 50, 1024])
        variable_summaries(W_fc1)
    with tf.name_scope('biases'):
        b_fc1 = bias_variable([1024])
        variable_summaries(b_fc1)
    with tf.name_scope('matmul_relu'):
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*50])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

with tf.name_scope('activations'):
    with tf.name_scope('weights'):
        W_fc2 = weight_variable([1024, 10])
        variable_summaries(W_fc2)
    with tf.name_scope('bias'):
        b_fc2 = bias_variable([10])
        variable_summaries(b_fc2)
    with tf.name_scope('matmul'):
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

with tf.name_scope('test_images'):
    image_shaped_input = tf.placeholder(tf.float32, [None, 256, 256, 1], name='image_shaped_input')
    tf.summary.image('input', image_shaped_input, 10)
    
with tf.name_scope('optimization'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_target, logits=y_conv), 
        name='cross_entropy'
    )
    tf.summary.scalar('cross_entropy', cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('evaluation'):
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_target,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    tf.summary.scalar('accuracy', accuracy)

In [6]:
tf.add_to_collection('x', x)
tf.add_to_collection('keep_prob', keep_prob)
tf.add_to_collection('y_conv', y_conv)
tf.add_to_collection('y_target', y_target)
tf.add_to_collection('accuracy', accuracy)

In [ ]:
merged = tf.summary.merge_all()
saver = tf.train.Saver(tf.trainable_variables())
sess = tf.InteractiveSession()

train_writer = tf.summary.FileWriter(train_dir + '/logs/train', sess.graph)
test_writer = tf.summary.FileWriter(train_dir + '/logs/test')

In [ ]:
sess.run(tf.global_variables_initializer())
test_batch_xs, test_batch_ys = mnist.test.next_batch(1000)

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_target: batch_ys, keep_prob: 0.5})
    train_labels = sess.run(y_conv, feed_dict={x: batch_xs, keep_prob: 1.0})
    train_summary = sess.run(
        merged, 
        feed_dict={
            x: batch_xs,
            y_target: batch_ys,
            keep_prob: 1.0,
            image_shaped_input: draw_labels(batch_xs, train_labels)
        }
    )

    if i % 50 == 49:
        os.makedirs(train_dir, exist_ok=True)
        checkpoint_path = '%s/models/mnist.ckpt' % train_dir
        saver.save(sess, checkpoint_path, global_step=i)
        
        train_writer.add_summary(train_summary, i)
        test_labels, acc = sess.run(
            [y_conv, accuracy], 
            feed_dict={
                x: test_batch_xs, 
                y_target: test_batch_ys,
                keep_prob: 1.0
            }
        )
        print('Test accuracy #%i: %f' % (i, acc))
        test_summary = sess.run(
            merged, 
            feed_dict={
                x: test_batch_xs,
                y_target: test_batch_ys,
                keep_prob: 1.0,
                image_shaped_input: draw_labels(test_batch_xs, test_labels)
            }
        )
        test_writer.add_summary(test_summary, i)

Test accuracy #49: 0.742000
Test accuracy #99: 0.858000
Test accuracy #149: 0.908000
Test accuracy #199: 0.923000
Test accuracy #249: 0.936000
Test accuracy #299: 0.945000
Test accuracy #349: 0.944000
Test accuracy #399: 0.951000
Test accuracy #449: 0.949000
Test accuracy #499: 0.957000
Test accuracy #549: 0.957000
Test accuracy #599: 0.958000
